# 1. Starting point

Input: (256, 256, 3) Chest X-ray image 566개
<br>
Label: 좌우 lung 마스킹되어 있는 흑백 이미지 566개
<br><br>
이미지 기준 좌측 폐: Right lung
이미지 기준 우측 폐: Left lung
<br>
※ Task<br>
&nbsp;&nbsp;&nbsp;    - Left lung, Right lung 분할(2 classes + 1 class(background)) <br>
&nbsp;&nbsp;&nbsp;    - 훈련 & 추론 + 후처리(보간법) 노이즈 제거(예측 성능 ↑)

## 데이터 구조

data <br>
&nbsp;&nbsp;    └ image/ <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;        resize_CHNCXR_0001_0.png <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;        resize_CHNCXR_0002_0.png <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;        resize_CHNCXR_0003_0.png <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;        ... <br>
&nbsp;&nbsp;    └ label/                     -> 사전 작업 후에는 사용하지 않습니다. <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;        resize_CHNCXR_0001_0.png <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;        resize_CHNCXR_0002_0.png <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;        resize_CHNCXR_0003_0.png <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;        ...    
&nbsp;&nbsp;    └ label_rl/ <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;               └ r/ <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;                    resize_CHNCXR_0001_0.png <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;                    resize_CHNCXR_0002_0.png <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;                    resize_CHNCXR_0003_0.png <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;                    ... <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;               └ l/ <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;                    resize_CHNCXR_0001_0.png <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;                    resize_CHNCXR_0002_0.png <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;                    resize_CHNCXR_0003_0.png <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;                    ...

## 사전 작업

label 폴더의 마스킹 이미지를 각각 좌측 폐('label_rl/l/*'), 우측 폐'label_rl/r/*'로 분할하여 어노테이션 처리하였습니다.<br>
추후 작업 시 라벨링을 배경: 0, 좌측 폐: 1, 우측 폐: 2로 하고 다시 one-hot encoding 작업으로 분할하여 진행한다.

# 2. Library Import

In [2]:
import os
import time
import datetime
import pickle
import statistics
from tqdm import tqdm
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.layers import Add, Input, Dense, Conv2D, Flatten, MaxPool2D, UpSampling2D
from tensorflow.keras.layers import Conv2DTranspose, Concatenate, BatchNormalization
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# 3. Data Preprocessing

### 3-3-1. Seed Fix

In [5]:
# fix random seed for reproductibility
seed = 777
np.random.seed(seed)
tf.random.set_seed(seed)

### 3-3-2. Hyper parameters

In [6]:
# hyper parameter
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 3
N_CLASSES = 2
EPOCHS = 80
BATCH_SIZE = 32

In [ ]:
image_path = './data/image/.png' # (256, 256, 3)
label_path = './data/label/.png' # (256, 256, 3)

In [6]:
# # sample
# image_cv2 = cv2.imread('./data/image/resize_CHNCXR_0001_0.png')
# label_cv2 = cv2.imread('./data/label/resize_CHNCXR_0001_0.png')
# print(image_cv2.shape) # (256, 256, 3)
# print(label_cv2.shape) # (256, 256, 3)

# cv2.imshow('image_cv2', image_cv2)
# cv2.imshow('label_cv2', label_cv2)

# cv2.waitKey(0)
# cv2.destroyAllWindows()

(256, 256, 3)
(256, 256, 3)


In [62]:
# label_gray = cv2.imread('./data/label/resize_CHNCXR_0001_0.png', cv2.IMREAD_GRAYSCALE)
label_gray = cv2.imread('./data/label/resize_CHNCXR_0075_0.png', cv2.IMREAD_GRAYSCALE)
print(label_gray.shape) # (256, 256)

cv2.imshow('label_gray', label_gray)

cv2.waitKey(0)
cv2.destroyAllWindows()

(256, 256)


- (256, 256) 이미지에서 가로 스캔이 먼저 되는 것(배경 제외)이 100의 픽셀 값을 가진다. 

In [118]:
np.random.seed(777)

cnt, labels = cv2.connectedComponents(label_gray)
# print(cnt) # 3
# print(labels.shape) # (256, 256)

ambiguous_count = 0
mass_a = np.zeros_like(label_gray)
mass_b = np.zeros_like(label_gray)

for i in range(cnt):
    # mass[labels==i] = [int(j) for j in np.random.randint(0, 255, 1)] # list 값 1개
    if i == 0:
        mass_a[labels==i] = [0] # list 값 1개
        mass_b[labels==i] = [0] # list 값 1개
    elif i == 1:
        mass_a[labels==i] = [100] # list 값 1개
        mass_b[labels==i] = [100] # list 값 1개
    elif i == 2:
        mass_a[labels==i] = [200] # list 값 1개
        mass_b[labels==i] = [200] # list 값 1개
    else:
        mass_a[labels==i] = [0]
        mass_b[labels==i] = [0]
        print(f"ambiguous pixel count: {ambiguous_count}")
        ambiguous_count += 1
                       
print(np.unique(mass_a, return_counts=True))
print(mass_a.shape)
# (array([ 47,  59, 103], dtype=uint8), array([ 8209,  7871, 49456], dtype=int64))
# (256, 256)

# opencv 시각화
cv2.imshow('label_gray', label_gray)
cv2.imshow('mass_a', mass_a)
cv2.waitKey(0)
cv2.destroyAllWindows()

(array([  0, 100, 200], dtype=uint8), array([55498,  4870,  5168], dtype=int64))
(256, 256)


In [119]:
mass_a_info = np.unique(mass_a, return_counts=True)
mass_b_info = np.unique(mass_b, return_counts=True)

a_index = np.delete(mass_a_info[0], np.argmax(mass_a_info[1]))
print(a_index)
b_index = np.delete(mass_b_info[0], np.argmax(mass_b_info[1]))
print(a_index)

a_pixel_volume = np.delete(mass_a_info[1], np.argmax(mass_a_info[1]))
print(a_pixel_volume)
b_pixel_volume = np.delete(mass_b_info[1], np.argmax(mass_b_info[1]))
print(b_pixel_volume)

mass_a[mass_a==a_index[np.argmax(a_pixel_volume)]] = 1
mass_a[mass_a!=1] = 0

mass_b[mass_b==b_index[np.argmin(b_pixel_volume)]] = 1
mass_b[mass_b!=1] = 0

[100 200]
[100 200]
[4870 5168]
[4870 5168]


In [123]:
print(mass_a.shape)
print(mass_b.shape)

# mass_a_count = mass_a.flatten()
# mass_b_count = mass_b.flatten()
# print(mass_a_count.shape, mass_a_count)
# print(mass_b_count.shape, mass_b_count)
print(np.unique(mass_a, return_counts=True))
print(np.unique(mass_b, return_counts=True))

(256, 256)
(256, 256)
(array([0, 1], dtype=uint8), array([60368,  5168], dtype=int64))
(array([0, 1], dtype=uint8), array([60666,  4870], dtype=int64))


- 좌우 구분 어떻게 하지?

In [147]:
print(mass_a[:, :128].shape)
print(mass_a[:, :128])
print(mass_a[:, 128:].shape)
print(mass_a[:, 128:])

print(np.unique(mass_a[:, 128:], return_counts=True))
print(np.unique(mass_a[:, :128], return_counts=True))

(256, 128)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(256, 128)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(array([0, 1], dtype=uint8), array([27600,  5168], dtype=int64))
(array([0], dtype=uint8), array([32768], dtype=int64))


In [154]:
# 이미지 상으로는 좌측에 있는 것이 ~right~, 우측에 있는 것이 ~left~ 변수 형태로 선언한다.
mass_a_left_info = np.unique(mass_a[:, 128:], return_counts=True)
mass_a_right_info = np.unique(mass_a[:, :128], return_counts=True)
print(mass_a_left_info)
print(mass_a_right_info)
# print(np.argmax(mass_a_left_info[1]))

print(mass_a_left_info[0])
print(type(mass_a_left_info[0]))

print(list(mass_a_left_info[0]))


if 1 not in mass_a_left_info[0]:
    print("좌폐 없음")

elif 1 not in mass_a_right_info[0]:
    print("우폐 없음")
    mask_l = mass_a
    
else:
    mask_a_left_info[# 픽셀 개수 비교


print(mask_l)
print(mask_l.shape)

(array([0, 1], dtype=uint8), array([27600,  5168], dtype=int64))
(array([0], dtype=uint8), array([32768], dtype=int64))
[0 1]
<class 'numpy.ndarray'>
[0, 1]
우폐 없음
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(256, 256)


In [153]:
mask_a_left_info[mask_a_left_info==1]

NameError: name 'mask_a_left_info' is not defined

In [131]:
print(mass_b[:, :128].shape)
print(mass_b[:, :128])
print(mass_b[:, 128:].shape)
print(mass_b[:, 128:])

print(np.unique(mass_b[:, :128], return_counts=True))
print(np.unique(mass_b[:, 128:], return_counts=True))

(256, 128)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(256, 128)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(array([0, 1], dtype=uint8), array([27898,  4870], dtype=int64))
(array([0], dtype=uint8), array([32768], dtype=int64))


In [ ]:
print(mass_a

In [ ]:
[100 200]
[4870 5168]

In [ ]:
0~5
0 1 2 3 4 5
0 + 5 / 2 = 2.5
-> 3

In [58]:
a = np.array([0, 2, 4, 6, 8, 10])
b = [0, 2, 4, 6, 8, 10]
print(a[:3])
print(b[:3])

[0 2 4]
[0, 2, 4]
